<table>
        <tr>
            <th>Nama</th>
            <th>NIM</th>
        </tr>
        <tr>
            <td>Ananda Ayu Sekar Wiranti</td>
            <td>2041720140</td>
        </tr>
        <tr>
            <td>M. Favian Apta Syahputra</td>
            <td>2041720189</td>
        </tr>
        <tr>
            <td>Ony Novianti</td>
            <td>2041720029</td>
        </tr>
        <tr>
            <td>Raihan Hidayatullah Djunaedi</td>
            <td>2041720108</td>
        </tr>
        <tr>
            <td>Satria Alief Pratama Sofyan</td>
            <td>2041720151</td>
        </tr>
        <tr>
            <td>Dony Kurniawan Nurcahyo</td>
            <td>2041720185</td>
        </tr>
  </table>

> ### 1. *Scrapping* data twitter pada akun @collegemenfess menggunakan library **Twint**

In [1]:
# Install Package
%pip install twint
%pip install nest_asyncio
%pip install snscrape

# Fix Issue with twint 20.1.20 from pip --> upgrade to 20.1.21 directly from git
%pip install --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 59.1/59.1 kB 447.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


Obtaining twint from git+https://github.com/twintproject/twint.git@origin/master#egg=twint
  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to d:\politeknik negeri malang\ti-3a\semester 5\pembelajaran mesin\minggu 16 - uas\uas fix\src\twint
  Resolved https://github.com/twintproject/twint.git to commit origin/master
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: twint
    Found existing installation: twint 2.1.21
    Uninstalling twint-2.1.21:
      Successfully uninstalled twint-2.1.21
  Running setup.py develop for twint


  Running command git clone --filter=blob:none --quiet https://github.com/twintproject/twint.git 'd:\Politeknik Negeri Malang\TI-3A\Semester 5\Pembelajaran Mesin\Minggu 16 - UAS\Uas Fix\src\twint'
  Running command git checkout -q origin/master


In [167]:
import twint
import nest_asyncio
nest_asyncio.apply()

# inisiasi obyek
c = twint.Config()

# config scrap
c.Username = "collegemenfess"
c.Since = "2022-12-05"
c.Until = "2022-12-11"
c.Limit = 5000
c.Store_csv = True
c.Output = "colle_scrap.csv"
c.Hide_output = True

# run scrap
twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [125]:
import numpy as np
import pandas as pd
import string
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Global Parameters
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [126]:
# Load dataset
dataset = pd.read_csv('colle_scrap.csv')
dataset.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1601728382596964352,1601728382596964352,2022-12-11 06:59:39 SE Asia Standard Time,2022-12-11,06:59:39,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1601727590469111808,1601727590469111808,2022-12-11 06:56:30 SE Asia Standard Time,2022-12-11,06:56:30,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1601726312796033024,1601726312796033024,2022-12-11 06:51:26 SE Asia Standard Time,2022-12-11,06:51:26,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1601726216561491968,1601726216561491968,2022-12-11 06:51:03 SE Asia Standard Time,2022-12-11,06:51:03,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1601725424429453313,1601725424429453313,2022-12-11 06:47:54 SE Asia Standard Time,2022-12-11,06:47:54,700,1046084008742801408,collegemenfess,COLLE | CEK PINNED😎,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


#### Preprocesing

In [127]:
# menghapus colom yang tidak digunakan
dataset = dataset.drop(dataset.iloc[:,0:10], axis=1)
dataset = dataset.drop(dataset.iloc[:,1:], axis=1)
dataset.head(10)

,tweet
0,[cm] bisa ga sih skripsi tuh diilangin aja 🥲
1,[cm] tolong infonya ya makasih guys https://t...
2,[cm] gaiss ada yg tau cara nyari jumlah atau d...
3,Besok aku praktik pertama kalinya di rumah sak...
4,ayooo bantu sender nyetok bahan2!! tia [cm] h...
5,[cm] spek laptop gini buat anak arsi cukup gak...
6,[cm] kira2 sopan gak ya ngechat dosbing hari i...
7,"Siapa yang pernah di posisi ini, ada solusi ga..."
8,[Cm] ada mahasiswa magister akuntansi/ekonomi ...
9,[cm] Gais kampus pendidikan yg terkenal di Ind...


In [128]:
#CASE FOLDING :Pengubahan huruf besar ke huruf kecil 
def clean_lower(lwr):
    lwr = lwr.lower() # mengubah menjadi lowercase text
    return lwr

# Tokenizing : Penghapusan url
def clean_url(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+|@\S+", '', tweet, flags=re.MULTILINE)
    return tweet

#Tokenizing : penghapusan karakter (@, '', "", dll) dan angka
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;][c][m]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

#Tokenizing : Penghapusan Spasi yang berlebihan 
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Tokenizing : Penghapusan number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

In [129]:
# Membuat kolom baru untuk hasil konten yang telah dilakukan preprocesing
dataset['content'] = dataset['tweet'].apply(clean_lower).apply(clean_url).apply(clean_punct).apply(_normalize_whitespace).apply(remove_number)

dataset.head()

,tweet,content
0,[cm] bisa ga sih skripsi tuh diilangin aja 🥲,bisa ga sih skripsi tuh diilangin aja
1,[cm] tolong infonya ya makasih guys https://t...,tolong infonya ya makasih guys
2,[cm] gaiss ada yg tau cara nyari jumlah atau d...,gaiss ada yg tau cara nyari jumlah atau daftar...
3,Besok aku praktik pertama kalinya di rumah sak...,besok aku praktik pertama kalinya di rumah sak...
4,ayooo bantu sender nyetok bahan2!! tia [cm] h...,ayooo bantu sender nyetok bahan tia


#### 2. Model Clustering berdasarkan data (positif, netral, negatif)

In [130]:
# membuat kolom baru, dan melakukan encode untuk mengonvert dataset ke numerik
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
content = encode.fit_transform(dataset['content'])
dataset_content = pd.DataFrame(content)

dataset['encode content'] = dataset_content

# cek data
dataset.head()

,tweet,content,encode content
0,[cm] bisa ga sih skripsi tuh diilangin aja 🥲,bisa ga sih skripsi tuh diilangin aja,837
1,[cm] tolong infonya ya makasih guys https://t...,tolong infonya ya makasih guys,4641
2,[cm] gaiss ada yg tau cara nyari jumlah atau d...,gaiss ada yg tau cara nyari jumlah atau daftar...,1395
3,Besok aku praktik pertama kalinya di rumah sak...,besok aku praktik pertama kalinya di rumah sak...,793
4,ayooo bantu sender nyetok bahan2!! tia [cm] h...,ayooo bantu sender nyetok bahan tia,693


In [131]:
# menentukan colom mana yang digunakan sebagai features untuk data scaling
dataset_columns = pd.DataFrame({
    'encode content': dataset['encode content']
})

col_names = dataset_columns.columns
features = dataset_columns[col_names]

In [132]:
# scalling data
from sklearn.preprocessing import StandardScaler

# proses scaling berdasarkan features yang telah ditentukan
scaler = StandardScaler()
scaler.fit(features.values)
dataset_scaled = scaler.transform(features.values)

# hasil dari scalling nantinya akan ditampung di variabel dataset_scaled
dataset_scaled = pd.DataFrame(dataset_scaled, columns=col_names)

In [133]:
# Menentukkan jumlah cluster menggunakan library k-means
from sklearn.cluster import KMeans

# n nya 3 karena ada 3 jenis komentar yaitu positif,negatif,netral. 
km = KMeans(n_clusters=3)
y_predicted = km.fit_predict(dataset_scaled[['encode content']])
y_predicted

# untuk menampung hasil clustering
dataset_columns['jenis_Komentar'] = y_predicted

In [134]:
# Membuat fungsi untuk menentukan jenis komentar
dataset1 = dataset_columns[dataset_columns.jenis_Komentar==0]
dataset2 = dataset_columns[dataset_columns.jenis_Komentar==1]
dataset3 = dataset_columns[dataset_columns.jenis_Komentar==2]

# menentukkan jenis komentar yang berniali 0 itu positif , 1 negatif dan 2 netral
conditions = [
    (dataset_columns['jenis_Komentar']==0),
    (dataset_columns['jenis_Komentar']==1),
    (dataset_columns['jenis_Komentar']==2)]
choices = ['Positif','Negativ','Netral']
dataset_columns['jenis_Komentar'] = np.select(conditions, choices)

In [135]:
# menambahkan kolom baru yaitu jenis_komentar dimana isi datanya mengambil dari hasil clusteriing  
dataset['jenis_komentar'] = dataset_columns['jenis_Komentar']

dataset = pd.DataFrame({
    'content': dataset['content'],
    'jenis_komentar': dataset['jenis_komentar']
})

# cek data 
dataset.head()

,content,jenis_komentar
0,bisa ga sih skripsi tuh diilangin aja,Positif
1,tolong infonya ya makasih guys,Negativ
2,gaiss ada yg tau cara nyari jumlah atau daftar...,Positif
3,besok aku praktik pertama kalinya di rumah sak...,Positif
4,ayooo bantu sender nyetok bahan tia,Positif


### 3. Labeling sesuai sentiment yang telah ditentukan

In [136]:
# Memilih Data mana yang akan digunakan
dataset = pd.DataFrame({
    'content' : dataset['content'],
    'jenis_komentar' : dataset['jenis_komentar']
})
dataset.head()

,content,jenis_komentar
0,bisa ga sih skripsi tuh diilangin aja,Positif
1,tolong infonya ya makasih guys,Negativ
2,gaiss ada yg tau cara nyari jumlah atau daftar...,Positif
3,besok aku praktik pertama kalinya di rumah sak...,Positif
4,ayooo bantu sender nyetok bahan tia,Positif


In [137]:
new_cols = {
    'jenis_komentar' : 'Labels',

}
#rename nama kolom jenis_komentar
dataset = dataset.rename(columns=new_cols)
dataset.head()

,content,Labels
0,bisa ga sih skripsi tuh diilangin aja,Positif
1,tolong infonya ya makasih guys,Negativ
2,gaiss ada yg tau cara nyari jumlah atau daftar...,Positif
3,besok aku praktik pertama kalinya di rumah sak...,Positif
4,ayooo bantu sender nyetok bahan tia,Positif


In [138]:
# Mengubah data kategorik menjadi data numerik 
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
dataset['Labels'] = encode.fit_transform(dataset['Labels'])

# cek data
dataset.head()

,content,Labels
0,bisa ga sih skripsi tuh diilangin aja,2
1,tolong infonya ya makasih guys,0
2,gaiss ada yg tau cara nyari jumlah atau daftar...,2
3,besok aku praktik pertama kalinya di rumah sak...,2
4,ayooo bantu sender nyetok bahan tia,2


In [139]:
# Memisahkan Fitur dengan Label
X = dataset['content']
y = dataset['Labels']

In [140]:
# Split data training dan testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [162]:
# Ekstrasi Fitur menggunakan Bag of Words dengan fungsi CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer(stop_words='english')

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
X_test = bow.transform(X_test)

### 4. Model Klasifikasi beserta evaluasi model

##### Decision Tree

In [163]:
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.metrics import accuracy_score, classification_report

# Pembuatan model Decision Tree Classifier
dt = DecisionTreeClassifier()

# Sesuaikan dataset ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

# Print hasil classification report
acc_rf = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")
print(classification_report(y_test, y_pred_dt))

Test set accuracy: 0.43
Test set accuracy: 0.43
              precision    recall  f1-score   support

           0       0.42      0.47      0.45        36
           1       0.44      0.44      0.44        32
           2       0.43      0.38      0.40        32

    accuracy                           0.43       100
   macro avg       0.43      0.43      0.43       100
weighted avg       0.43      0.43      0.43       100



##### Multinomial NB

In [164]:
#TRAINING DAN EVALUASI MODEL
#Model yang saya gunakan adalah Algoritma Naive Bayes dengan Menggunakan fungsi MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.96
Hasil akurasi data test: 0.41


### 5. Prediksi sentimen data twitter

In [168]:
import twint
import nest_asyncio
nest_asyncio.apply()

# inisiasi obyek
c = twint.Config()

# config scrap
c.Username = "collegemenfess"
# c.Since = "2022-12-04 00:00:00"
# c.Until = "2022-12-11 00:00:00"
c.Limit = 500
c.Store_csv = True
c.Output = "colle_scrap1.csv"
c.Hide_output = True

# run scrap
twint.run.Search(c)

In [144]:
# load data baru
dataset = pd.read_csv('colle_scrap1.csv')

In [145]:
# menghapus colom yang tidak digunakan
dataset = dataset.drop(dataset.iloc[:,0:10], axis=1)
dataset = dataset.drop(dataset.iloc[:,1:], axis=1)
dataset.head(10)

,tweet
0,[cm] referensiku tahun 2013. pas dicek di sint...
1,"[cm] yakinin aku, aku bisa sempro online sendi..."
2,"[cm] guys mau nanya, kalo pengutipan kayak ""me..."
3,[cm] deket2 tahun baru dosen mulai cuti dari t...
4,Mau minta pendapatnya dong [cm] lagi bimbang ...
5,"Mau tanya dong, menurut kalian standar ip k b..."
6,[Cm] besok mau sidang akhir deg degan banget g...
7,"tadi aku stalk search bar tentang sem-pls, tru..."
8,"[cm] kalian kalo liburan, tetep bayaar kosan n..."
9,[cm] kalian yg skripsian dapet dospem yg kille...


In [146]:
# Membuat kolom baru untuk hasil konten yang telah dilakukan preprocesing
dataset['content'] = dataset['tweet'].apply(clean_lower).apply(clean_url).apply(clean_punct).apply(_normalize_whitespace).apply(remove_number)

dataset.head()

,tweet,content
0,[cm] referensiku tahun 2013. pas dicek di sint...,referensiku tahun pas dicek di sinta udh s ta...
1,"[cm] yakinin aku, aku bisa sempro online sendi...",yakinin aku aku bisa sempro online sendirian m...
2,"[cm] guys mau nanya, kalo pengutipan kayak ""me...",guys mau nanya kalo pengutipan kayak menurut x...
3,[cm] deket2 tahun baru dosen mulai cuti dari t...,deket tahun baru dosen mulai cuti dari tgl brp ya
4,Mau minta pendapatnya dong [cm] lagi bimbang ...,mau minta pendapatnya dong lagi bimbang banget


In [147]:
# Clustering
# membuat kolom baru, dan melakukan encode untuk mengonvert dataset ke numerik
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
content = encode.fit_transform(dataset['content'])
dataset_content = pd.DataFrame(content)

dataset['encode content'] = dataset_content

# cek data
dataset.head()

,tweet,content,encode content
0,[cm] referensiku tahun 2013. pas dicek di sint...,referensiku tahun pas dicek di sinta udh s ta...,414
1,"[cm] yakinin aku, aku bisa sempro online sendi...",yakinin aku aku bisa sempro online sendirian m...,485
2,"[cm] guys mau nanya, kalo pengutipan kayak ""me...",guys mau nanya kalo pengutipan kayak menurut x...,194
3,[cm] deket2 tahun baru dosen mulai cuti dari t...,deket tahun baru dosen mulai cuti dari tgl brp ya,107
4,Mau minta pendapatnya dong [cm] lagi bimbang ...,mau minta pendapatnya dong lagi bimbang banget,359


In [148]:
# menentukan colom mana yang digunakan sebagai features untuk data scaling
dataset_columns = pd.DataFrame({
    'encode content': dataset['encode content']
})

col_names = dataset_columns.columns
features = dataset_columns[col_names]

# scalling data
from sklearn.preprocessing import StandardScaler

# proses scaling berdasarkan features yang telah ditentukan
scaler = StandardScaler()
scaler.fit(features.values)
dataset_scaled = scaler.transform(features.values)

# hasil dari scalling nantinya akan ditampung di variabel dataset_scaled
dataset_scaled = pd.DataFrame(dataset_scaled, columns=col_names)

# Menentukkan jumlah cluster menggunakan library k-means
from sklearn.cluster import KMeans

# disini kami tentukkan n nya 3 karena ada 3 jenis komentar yaitu positif.negatif,netral. 
km = KMeans(n_clusters=3)
y_predicted = km.fit_predict(dataset_scaled[['encode content']])
y_predicted

# untuk menampung hasil clustering
dataset_columns['jenis_Komentar'] = y_predicted

# Membuat fungsi untuk menentukan jenis komentar
dataset1 = dataset_columns[dataset_columns.jenis_Komentar==0]
dataset2 = dataset_columns[dataset_columns.jenis_Komentar==1]
dataset3 = dataset_columns[dataset_columns.jenis_Komentar==2]

# menentukkan jenis komentar yang berniali 0 itu positif , 1 negatif dan 2 netral
conditions = [
    (dataset_columns['jenis_Komentar']==0),
    (dataset_columns['jenis_Komentar']==1),
    (dataset_columns['jenis_Komentar']==2)]
choices = ['Positif','Negativ','Netral']
dataset_columns['jenis_Komentar'] = np.select(conditions, choices)

# menambahkan kolom baru yaitu jenis_kolom komentar dimana isi datanya mengambil dari hasil clusteriing  
dataset['jenis_komentar'] = dataset_columns['jenis_Komentar']

dataset = pd.DataFrame({
    'content': dataset['content'],
    'jenis_komentar': dataset['jenis_komentar']
})
dataset.head()

,content,jenis_komentar
0,referensiku tahun pas dicek di sinta udh s ta...,Negativ
1,yakinin aku aku bisa sempro online sendirian m...,Negativ
2,guys mau nanya kalo pengutipan kayak menurut x...,Netral
3,deket tahun baru dosen mulai cuti dari tgl brp ya,Positif
4,mau minta pendapatnya dong lagi bimbang banget,Negativ


In [149]:
# Labeling
# Memilih Data mana yang akan digunakan
dataset = pd.DataFrame({
    'content' : dataset['content'],
    'jenis_komentar' : dataset['jenis_komentar']
})

new_cols = {
    'jenis_komentar' : 'Labels',

}
#rename nama kolom jenis_komentar
dataset = dataset.rename(columns=new_cols)

dataset.head()

,content,Labels
0,referensiku tahun pas dicek di sinta udh s ta...,Negativ
1,yakinin aku aku bisa sempro online sendirian m...,Negativ
2,guys mau nanya kalo pengutipan kayak menurut x...,Netral
3,deket tahun baru dosen mulai cuti dari tgl brp ya,Positif
4,mau minta pendapatnya dong lagi bimbang banget,Negativ


In [150]:
# Mengubah data kategorik menjadi data numerik 
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
dataset['Labels'] = encode.fit_transform(dataset['Labels'])

# cek data
dataset.head()

,content,Labels
0,referensiku tahun pas dicek di sinta udh s ta...,0
1,yakinin aku aku bisa sempro online sendirian m...,0
2,guys mau nanya kalo pengutipan kayak menurut x...,1
3,deket tahun baru dosen mulai cuti dari tgl brp ya,2
4,mau minta pendapatnya dong lagi bimbang banget,0


In [151]:
# Memisahkan Fitur dengan Label
X = dataset['content']
y = dataset['Labels']

In [152]:
# Split data training dan testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [159]:
# Ekstrasi Fitur menggunakan Bag of Words dengan fungsi CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer(stop_words='english')

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
X_test = bow.transform(X_test)

X_train.shape
# X_test.shape

(400, 2136)

In [165]:
# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

# Print hasil classification report
acc_rf = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")
print(classification_report(y_test, y_pred_dt))

Test set accuracy: 0.43
Test set accuracy: 0.43
              precision    recall  f1-score   support

           0       0.42      0.47      0.45        36
           1       0.44      0.44      0.44        32
           2       0.43      0.38      0.40        32

    accuracy                           0.43       100
   macro avg       0.43      0.43      0.43       100
weighted avg       0.43      0.43      0.43       100



In [166]:
# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.96
Hasil akurasi data test: 0.41
